In [1]:
from pLM_KD import EsmForMaskedLM  # or use the relevant import if different

teacher_model_path = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/esm_150m_ecoli_finetuning_1/checkpoint-19000"
model = EsmForMaskedLM.from_pretrained(teacher_model_path)

# Print total number of parameters (both trainable and non-trainable)
total_parameters = model.num_parameters()
print(f"Total parameters: {total_parameters}")


2025-04-08 21:29:39.587331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-08 21:29:39.603972: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-08 21:29:39.604015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 21:29:39.616609: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-08 21:29:40.976324: W tensorflow/compiler/tf

Total parameters: 149843867


In [2]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")


Trainable parameters: 0


In [7]:
from pLM_KD import EsmForMaskedLM, EsmTokenizer, get_peft_model, LoraConfig, TaskType
import torch

# Path to your fine-tuned teacher model checkpoint
teacher_model_path = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/esm_150m_ecoli_finetuning_1/checkpoint-19000"
base_model = "facebook/esm2_t30_150M_UR50D"

# Load your model and tokenizer. If you have a separate base model from which to load the tokenizer,
# adjust accordingly (here I assume the tokenizer is in the same location for simplicity).
model = EsmForMaskedLM.from_pretrained(teacher_model_path)
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")

# Before applying LoRA, make sure the entire model requires gradients (optional if you want to inspect them)
for param in model.parameters():
    param.requires_grad = True

# Determine the target modules. Here we assume "key" and "value" layers are the targets.
# You might want to include additional logic to determine these names dynamically.
target_module_names = ["key", "value"]

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # Adjust task type as needed, e.g., CAUSAL_LM for ESM models
    r=8,
    lora_alpha=32,
    target_modules=target_module_names,
    lora_dropout=0.1,
    bias="none"
)

# Apply LoRA integration. This wraps your model and adds extra trainable parameters.
model = get_peft_model(model, lora_config)

# Set which parameters should be trainable:
# For PEFT-style fine-tuning (e.g. LoRA), you typically set only the LoRA modules and sometimes the LM head to require gradients.
for name, param in model.named_parameters():
    if "lora" in name or "lm_head" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Optionally, print details of each trainable parameter for debugging
print("Trainable parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"- {name}: {param.shape} ({param.numel()} parameters)")

# Calculate total numbers and print summary
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nLoRA integration complete. Trainable parameters: {trainable_params} "
      f"({100 * trainable_params / total_params:.2f}% of total {total_params})")


2025-04-08 21:45:02,989 INFO: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!


Trainable parameters:
- base_model.model.esm.encoder.layer.0.attention.self.key.lora_A.default.weight: torch.Size([8, 640]) (5120 parameters)
- base_model.model.esm.encoder.layer.0.attention.self.key.lora_B.default.weight: torch.Size([640, 8]) (5120 parameters)
- base_model.model.esm.encoder.layer.0.attention.self.value.lora_A.default.weight: torch.Size([8, 640]) (5120 parameters)
- base_model.model.esm.encoder.layer.0.attention.self.value.lora_B.default.weight: torch.Size([640, 8]) (5120 parameters)
- base_model.model.esm.encoder.layer.1.attention.self.key.lora_A.default.weight: torch.Size([8, 640]) (5120 parameters)
- base_model.model.esm.encoder.layer.1.attention.self.key.lora_B.default.weight: torch.Size([640, 8]) (5120 parameters)
- base_model.model.esm.encoder.layer.1.attention.self.value.lora_A.default.weight: torch.Size([8, 640]) (5120 parameters)
- base_model.model.esm.encoder.layer.1.attention.self.value.lora_B.default.weight: torch.Size([640, 8]) (5120 parameters)
- base_mod

In [10]:
import torch
from peft import PeftModel
from pLM_KD import EsmForMaskedLM, EsmTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Your teacher model checkpoint path (LoRA fine-tuned model checkpoint)
teacher_model_path = "/home/sdowell/scratch/Thesis/BenchmarkingFinetuning/runs/esm_150m_ecoli_finetuning_1/checkpoint-19000"

# Instead of passing a string as base_model, load the base model object:
base_model_obj = EsmForMaskedLM.from_pretrained("facebook/esm2_t30_150M_UR50D").to(device)

# Load the LoRA fine-tuned teacher model using the PEFT utility
teacher_model = PeftModel.from_pretrained(base_model_obj, teacher_model_path)
teacher_model.eval()  # Set teacher to evaluation mode

# Optionally, load your tokenizer as needed.
tokenizer = EsmTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D")


In [11]:
teacher_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): EsmForMaskedLM(
      (esm): EsmModel(
        (embeddings): EsmEmbeddings(
          (word_embeddings): Embedding(33, 640, padding_idx=1)
          (dropout): Dropout(p=0.0, inplace=False)
          (position_embeddings): Embedding(1026, 640, padding_idx=1)
        )
        (encoder): EsmEncoder(
          (layer): ModuleList(
            (0-29): 30 x EsmLayer(
              (attention): EsmAttention(
                (self): EsmSelfAttention(
                  (query): Linear(in_features=640, out_features=640, bias=True)
                  (key): lora.Linear(
                    (base_layer): Linear(in_features=640, out_features=640, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=640, out_features=8, bias=False)
                    )
  